In [1]:
!pip install datasets langchain_community langchain_chroma langchain langchain_core tiktoken sentence-transformers==2.2.2 lark InstructorEmbedding bitsandbytes accelerate >> /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.8.2 requires keras-core, which is not installed.
keras-nlp 0.9.3 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.2 which is incompatible.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 23.2 which is incompatible.
jupyterlab 4.1.6 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
kfp 2.5

Загрузим датасет medal https://huggingface.co/datasets/bigbio/medal

In [2]:
!wget -nc -P data/ https://zenodo.org/record/4482922/files/full_data.csv.zip

--2024-05-06 11:54:14--  https://zenodo.org/record/4482922/files/full_data.csv.zip
Resolving zenodo.org (zenodo.org)... 188.184.98.238, 188.185.79.172, 188.184.103.159, ...
Connecting to zenodo.org (zenodo.org)|188.184.98.238|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/4482922/files/full_data.csv.zip [following]
--2024-05-06 11:54:15--  https://zenodo.org/records/4482922/files/full_data.csv.zip
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 5228583309 (4.9G) [application/octet-stream]
Saving to: 'data/full_data.csv.zip'

full_data.csv.zip   100%[===================>]   4.87G  30.0MB/s    in 2m 52s  

2024-05-06 11:57:07 (28.9 MB/s) - 'data/full_data.csv.zip' saved [5228583309/5228583309]



In [3]:
!unzip data/full_data.csv.zip

Archive:  data/full_data.csv.zip
  inflating: full_data.csv           


In [5]:
!cat full_data.csv | head

TEXT,LOCATION,LABEL
alphabisabolol has a primary antipeptic action depending on dosage which is not caused by an alteration of the phvalue the proteolytic activity of pepsin is reduced by percent through addition of bisabolol in the ratio of the antipeptic action of bisabolol only occurs in case of direct contact in case of a previous contact with the ATP the inhibiting effect is lost,56,substrate
a report is given on the recent discovery of outstanding immunological properties in ba ncyanoethyleneurea having a low molecular mass m experiments in ds CS bearing wistar rats have shown that ba at a dosage of only about percent ld mg kg and negligible lethality percent results in a REC rate of percent without hyperglycemia and in one test of percent with hyperglycemia under otherwise unchanged conditions the REF substance ifosfamide if a further development of cyclophosphamide applied without hyperglycemia in its most efficient dosage of percent ld mg kg brought about a recovery rate of pe

# Reading and indexing the data

In [6]:
!head -n2000 full_data.csv > full_data2000.csv

In [7]:
from langchain_community.document_loaders.csv_loader import CSVLoader

FILE_PATH = "full_data2000.csv" 
loader = CSVLoader(file_path=FILE_PATH)

data = loader.load()

In [8]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
import torch

emb_model = HuggingFaceInstructEmbeddings()

/opt/conda/lib/python3.10/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer


/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


max_seq_length  512


In [9]:
from langchain.vectorstores import Chroma

vectordb = Chroma.from_documents(data, emb_model)
vectordb.persist()

retriever = vectordb.as_retriever() 

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [10]:
#let's check that search on the documents works well
retriever.invoke("ceftobiprole bpr")

[Document(page_content='TEXT: cephacetrile a parenteral cephalosporin was evaluated for in vitro antibacterial activity clinical pharmacology and effectiveness in the treatment of severe infections the antibacterial activity against isolates was determined by an agardilution technique the mics were to mugml for GAS d pneumoniae and staph aureus mugml for e coli and klebsiellaenterobacter mugml for pr mirabilis and more than mugml for ps aeruginosa a few strains of klebsiella and e coli had mics of more than mcgml serum C2 after and g of im cephacetrile were respectively and mugml T3 hr and and mugml T3 hr serum levels after iv infusion of and g were respectively and mugml T3 hr and and mugml after hr urine C2 after and g im cephacetrile were respectively and mugml in the hr period and and mugml in the hr period renal clearance was mlmin m renal excretion was about of the dose hr after im injection cephacetrile was well tolerated when po im with lidocaine mild phlebitis occurred sometim

# Creating langchain chat model

In [11]:
!pip install fireworks-ai

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 2.2 MB/s eta 0:00:00


In [12]:
import os
os.environ["FIREWORKS_API_KEY"] = #your api-key

In [13]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
import torch
from langchain.chat_models import ChatFireworks


llm = ChatFireworks(api_key= ) #your api-key

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatFireworks` was deprecated in LangChain 0.0.26 and will be removed in 0.2. An updated version of the class exists in the langchain-fireworks package and should be used instead. To use it run `pip install -U langchain-fireworks` and import as `from langchain_fireworks import ChatFireworks`.
  warn_deprecated(


# Prompt Engineering

In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('fireworks-ai/firefunction-v1')
print(tokenizer.chat_template)

tokenizer_config.json:   0%|          | 0.00/3.18k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/50.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{%- set message_roles = ['SYSTEM', 'FUNCTIONS', 'USER', 'ASSISTANT', 'TOOL'] -%}
{%- set ns = namespace(seen_non_system=false, messages=messages, content='', functions=[]) -%}
{{ bos_token }}
{#- Basic consistency checks -#}
{%- if not ns.messages -%}
  {{ raise_exception('No messages') }}
{%- endif -%}
{%- if ns.messages[0]['role'] | upper != 'SYSTEM' -%}
  {%- set ns.messages = [{'role': 'SYSTEM', 'content': 'You are a helpful assistant with access to functions. Use them if required.'}] + ns.messages -%}
{%- endif -%}
{%- if ns.messages | length < 2 or ns.messages[0]['role'] | upper != 'SYSTEM' or ns.messages[1]['role'] | upper != 'FUNCTIONS' -%}
  {{ raise_exception('Expected either "functions" or ["system", "functions"] as the first messages') }}
{%- endif -%}
{%- for message in ns.messages -%}
  {%- set role = message['role'] | upper -%}
  {#- Validation -#}
  {%- if role not in message_roles -%}
    {{ raise_exception('Invalid role ' + message['role'] + '. Only ' + message_roles 

In [67]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.prompts.chat import MessagesPlaceholder


#whether we want to save history of the chat:
USE_HISTORY = True
instruction = "Use context to answer the question, using the following rules: Do not alter text that is in quotation marks. Be sure to add a link to the full document at the end url."

if USE_HISTORY:
    prompt = ChatPromptTemplate.from_messages([
        ("system", instruction),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}")
    ]
)
else:
    prompt = PromptTemplate(
    input_variables=["question"],
    template=instruction + "Question: {question} Answer:"
)

# Creating langchain pipeline

In [68]:
from langchain_core.runnables import RunnablePassthrough, RunnableSequence
from langchain_core.output_parsers.string import StrOutputParser
from langchain.chains import LLMChain

chain = prompt | llm

In [69]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

history = ChatMessageHistory()
chain_with_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: history,
    input_messages_key="question",
    history_messages_key="chat_history",
)

In [70]:
def query_llm(question):
    if USE_HISTORY:
        print(chain_with_history.invoke({'question': question},{"configurable": {"session_id": "unused"}}).content)
        return
    print(chain.invoke({'question': question}).content)

In [71]:
query_llm("What's Wilson's disease?")

Wilson's disease is a rare genetic disorder that affects the brain and nervous system. It is caused by a mutation in the ATP7B gene, which codes for a protein that helps to transport copper through the body. When this protein is mutated, copper builds up in the body's tissues, including the brain, liver, and kidneys, leading to a range of symptoms and complications.

According to the Wilson's Disease Association, the symptoms of Wilson's disease can vary in severity and may include:

* Abnormal movements, such as tremors, rigidity, and difficulty with coordination and balance
* Mental health problems, such as depression, anxiety, and dementia
* Vision problems, including blurred vision, double vision, and loss of peripheral vision
* Hearing problems, including loss of hearing or ringing in the ears
* Liver problems, including fatty liver, scarring, and liver failure
* Kidney problems, including kidney damage and failure
* Bone problems, including osteoporosis, osteopenia, and bone pain

In [72]:
query_llm("How to treat pneumonia?")

Pneumonia is a serious infection of the lungs that can be caused by bacteria, viruses, or fungi. Treatment for pneumonia depends on the cause of the infection and the severity of the symptoms. Here are some common ways to treat pneumonia:

1. Antibiotics: If the pneumonia is caused by bacteria, antibiotics are the primary treatment. The type of antibiotic used will depend on the type of bacteria causing the infection. It's important to complete the full course of antibiotics, even if symptoms improve before finishing the medication.
2. Fever reduction: To reduce fever, medications such as acetaminophen (Tylenol) or ibuprofen (Advil, Motrin) can be used. Never give aspirin to children under 18 years old, as it can increase the risk of Reye's syndrome, a rare but serious condition.
3. Rest: Rest is essential to help the body recover from pneumonia. Encourage the person to rest as much as possible, especially in the first few days after diagnosis.
4. Fluids: Drinking plenty of fluids can 

In [73]:
query_llm("What have I just asked you?")

You have just asked me: "How to treat pneumonia?".
